In [40]:
import pandas as pd 
import json
import sentifish
import requests
import urllib
import google_play_scraper
from sentifish import Sentiment
from google_play_scraper import app
from google_play_scraper import Sort, reviews_all

In [41]:
with open('Data.jl') as f:
    lines = f.read().splitlines()

In [42]:
df_inter = pd.DataFrame(lines)
df_inter.columns = ['json_element']

In [43]:
print(df_inter.columns)

Index(['json_element'], dtype='object')


In [44]:

df_inter['json_element'].apply(json.loads)

0       {'appId': 'com.casualgamefactory.wordapart', '...
1       {'appId': 'com.noodlecake.typeshift', 'appName...
2       {'appId': 'com.q42.ohno', 'appName': '0h n0', ...
3       {'appId': 'com.finalgamestudio.aurora.hex.jigs...
4       {'appId': 'za.co.twinc.a9letterjumble', 'appNa...
                              ...                        
9995    {'appId': 'com.simulationcurriculum.skysafari5...
9996    {'appId': 'com.vitotechnology.StarWalk', 'appN...
9997    {'appId': 'com.pa.lightpollutionmap', 'appName...
9998    {'appId': 'com.escapistgames.starchartgoogleed...
9999    {'appId': 'com.cgs.HiddenSurvivor', 'appName':...
Name: json_element, Length: 10000, dtype: object

In [45]:
df_final = pd.json_normalize(df_inter['json_element'].apply(json.loads))

In [46]:
#Zero Permission Apps from the take dataset
zeroPermissionappList=[]
for zeroApp in range(len(df_final)):
    if len(df_final['allPermissions'][zeroApp])==0:
        zeroPermissionappList.append(df_final['appId'][zeroApp])
print(zeroPermissionappList)
#This zero permission app that link to an external source or fake privacy policy permission

['com.hyperionics.avarLic', 'org.coolreader.donation.silver', 'com.Hololive_Stickers', 'com.ginkage.ejlookup', 'com.joseluisgalan.android.chatstats', 'info.casualcomputing.sputter', 'com.piomusic.piomusicv', 'de.smartchord.droid.plug.cof', 'com.bsoft.asadkhan.maqamatalarabiya', 'online.astrotools.miroir3', 'com.rmsoft.myemdr', 'com.vlvolad.quantumoscillator', 'com.cranf.android.deepware.rupture', 'com.battery.lifespan.extender', 'com.cranf.android.quickonf.battery.free', 'southbridge.led_calculatorpro', 'jackpal.droidexaminer', 'com.scappyapps.fourthirtytwo', 'com.zebproj.etherpad', 'com.iforpowell.android.ipbikekey', 'de.khe.ringrechner', 'com.MikaMobile.Battleheart', 'ru.wellapp.bukvica', 'com.simplesoftwarestudio.troparions', 'com.simplesoftwarestudio.molitvoslov', 'com.medclever.pocketdoctor_free', 'com.simplesoftwarestudio.psaltyr', 'com.ovirium.simoron', 'com.pasanen.belarusbirds', 'com.nappsislamic.znaniia', 'com.faultexception.reader.pro', 'com.Waseem.muhammadusama.GeoEngineeri

In [49]:
print(len(zeroPermissionappList))
zeroPermissionappDict={}

for app_id in zeroPermissionappList:
    try:
        getCategory = app(
        app_id,
        lang='en', # defaults to 'en'
        country='us' # defaults to 'us'
        )
        

        key=getCategory['title']
        value=getCategory['genre']
        print(key+"-"+value)
        #create a key value pair of app Category and app Name
        zeroPermissionappDict.update({key:value})
    except urllib.error.HTTPError:
        continue
    except google_play_scraper.exceptions.NotFoundError:
        continue
    except AttributeError:
        continue
    except ValueError:
        continue
    except requests.exceptions.HTTPError:
        continue
    except google_play_scraper.exceptions.ExtraHTTPError:
        continue

    

124
@Voice Premium License-Productivity
Cool Reader Silver Donation-Tools
Hololive Stickers-Communication
EJLookup — Japanese Dictionary-Books & Reference
Chat Stats for WhatsApp-Social
Sputter Music Sequencer-Music & Audio
PiOMuSiC-Education
s.mart Circle of Fifths-Books & Reference
Maqamat Al-Arabiya-Music & Audio
COMPLETE ASTROLOGICAL PORTRAIT (an Astral Mirror)-Lifestyle
My EMDR-Lifestyle
Quantum Harmonic Oscillator-Education
Deepware Rupture-Productivity
Battery Lifespan Extender-Tools
QuickONF Battery Widgets-Tools
LED Calculator Pro-Tools
Droid Examiner-Tools
Binaural Chakra Therapy-Health & Fitness
EtherSurface-Music & Audio
IpBikeKey-Health & Fitness
Shooting-Score-Calculator-Sports
Battleheart-Role Playing
Буквица древних Славян-Education
Тропари православные-Books & Reference
Молитвослов-Books & Reference
Первая помощь - Карманный доктор (базовая версия)-Medical
Псалтирь-Books & Reference
Симорон, Ритуалы, Желания-Lifestyle
Птицы Беларуси-Education
ОБЯЗАТЕЛЬНЫЕ ЗНАНИЯ ДЛЯ МУ

In [ ]:
# collect zero Permission app from Social and Communcation as my study......
socialAppList=[]
communicationAppList=[]
for name,category in zeroPermissionappDict.items():
    if(category=='Communication'):
        communicationAppList.append(name)
    if(category=='Social'):
        socialAppList.append(name)


### Black List Apps From Zero Permissions..

In [64]:
print("*****************Communication Apps**************")
for i in communicationAppList:
    print(i)
print("*****************Socail Apps*********************")
for j in socialAppList:
    print(j)

*****************Communication Apps**************
Hololive Stickers
ANT USB Service
*****************Socail Apps*********************
Chat Stats for WhatsApp


In [65]:
#Most Dangerous Permission in Playstore apps access
Dangerous=['directly call phone numbers','read call log','read phone status and identity',
          'read sensitive log data','retrieve running apps','delete all app cache data']
appList=[]
temp=[]
permissionList=[]


In [ ]:
#More Dangerous apps permission under other category is read sensitive log data

for i in range(len(df_final)):#from .jl jsonlines 
    #print(df_final['appName'][i])
    temp=df_final['allPermissions'][i]
    #print(temp)
    for j in range(len(temp)):
        #print(temp[j]['permission'],j)
        permissionList.append(temp[j]['permission'])
    if Dangerous[3] in permissionList:
        appList.append(df_final['appId'][i])
        permissionList=[]
    else:
        permissionList=[]
#print(appList)
new_appList = []
for elem in appList:
    if elem not in new_appList:
        new_appList.append(elem) 
for app_id in new_appList:
    getCategory = app(
    app_id,
    lang='en', # defaults to 'en'
    country='us' # defaults to 'us'
    )
    print(getCategory['genre']+"-"+getCategory['title'])

Productivity-LimeLife Goals App for Beauty Guides


In [21]:
for i in range(len(df_final)):#from .jl jsonlines 
    #print(df_final['appName'][i])
    temp=df_final['allPermissions'][i]
    #print(temp)
    for j in range(len(temp)):
        #print(temp[j]['permission'],j)
        permissionList.append(temp[j]['permission'])
    if Dangerous[0] in permissionList and Dangerous[1] in permissionList and Dangerous[2] in permissionList and Dangerous[3] in permissionList and Dangerous[4] in permissionList:
        appList.append(df_final['appId'][i])
        permissionList=[]
    else:
        permissionList=[]

In [22]:
#avoid duplicates elements
new_appList = []
for elem in appList:
    if elem not in new_appList:
        new_appList.append(elem) 

In [23]:
print(len(new_appList))
print(new_appList)
#Apps List that statisfy the above condintion

3
['ru.agc.acontactnexttrial', 'net.soti.mobicontrol.androidwork', 'net.dinglisch.android.taskerm']


In [28]:

for app_id in new_appList:
    getCategory = app(
    app_id,
    lang='en', # defaults to 'en'
    country='us' # defaults to 'us'
    )
    print(getCategory['genre']+"-"+getCategory['title'])

Communication-Phone and Contacts - AGContacts, Lite edition
Business-MobiControl | Android Enterprise
Tools-Tasker


In [32]:
for app_id in new_appList:
    result = reviews_all(
    app_id,
    sleep_milliseconds=0, # defaults to 0
    lang='en', # defaults to 'en'
    country='us', # defaults to 'us'
    sort=Sort.MOST_RELEVANT, # defaults to Sort.MOST_RELEVANT
    filter_score_with=None # defaults to None(means all score)
    )
    reviewList=[]
    for i in range(len(result)):
        reviewList.append(result[i]['content'])
#code for scrape the review of that app
print(reviewList)

['This is a powerful automation tool that gives you the ability to control almost any setting on your phone or tablet. It is combined with a wide variety of event triggers, custom dialogs, and simple programming tools to get your device to do exactly what you want to do when you want it. It\'s extremely useful for power users and invaluable for anyone with a little programming knowledge. - Want to reset your ringer volume when you wake up in the morning and silence it at night? This app can do that. - Want to disable your Wi-Fi when connected to your car\'s bluetooth to save battery? This app can do that. - Want your phone to activate it\'s GPS, take pictures every minute, and send those pictures and the phone\'s GPS coordinates whenever it receives a text reading "This phone is stolen" from a range of known trusted phone numbers? This app can do that. - Want fast support from a knowledgable developer? This app has that.', "Thanks for developing this app. It is really useful to my dail

In [33]:
print(len(reviewList))

10164


In [39]:
countPositive=0
countNegative=0
for eachReview in range(100):
    obj=Sentiment(reviewList[eachReview])
    polarity = obj.analyze()
    if obj.isPositive():
        countPositive+=1
    else:
        countNegative+=1
print(countPositive)
print(countNegative)

    
    

IndexError: tuple index out of range